In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})

response = response.additional_kwargs["function_call"]["arguments"]

import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What is the capital city of Italy?', 'answers': [{'answer': 'Rome', 'correct': True}, {'answer': 'Milan', 'correct': False}, {'answer': 'Naples', 'correct': False}, {'answer': 'Florence', 'correct': False}]}
{'question': 'Which ancient structure is known as the largest amphitheater ever built?', 'answers': [{'answer': 'The Colosseum', 'correct': True}, {'answer': 'The Pantheon', 'correct': False}, {'answer': 'The Forum', 'correct': False}, {'answer': 'The Circus Maximus', 'correct': False}]}
{'question': 'What river runs through Rome?', 'answers': [{'answer': 'Tiber', 'correct': True}, {'answer': 'Po', 'correct': False}, {'answer': 'Arno', 'correct': False}, {'answer': 'Adige', 'correct': False}]}
{'question': 'Which famous Roman general crossed the Rubicon River?', 'answers': [{'answer': 'Julius Caesar', 'correct': True}, {'answer': 'Augustus', 'correct': False}, {'answer': 'Pompey', 'correct': False}, {'answer': 'Nero', 'correct': False}]}
{'question': 'What is the name